In [ ]:
# ✅ STEP 1: Upload kaggle.json
from google.colab import files
files.upload()  # Upload your kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tavishapal","key":"deec013b2541c82ebaab0204a105178e"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ✅ STEP 3: Clean install YOLOv5 and dependencies

# Ensure you're in a good directory
%cd /content

# Remove previous yolov5 folder if corrupted
!rm -rf yolov5

# Clone the YOLOv5 repository fresh
!git clone --depth 1 https://github.com/ultralytics/yolov5

# Change to the yolov5 directory
%cd yolov5

# Install all required dependencies cleanly
!pip install -r requirements.txt


/content
Cloning into 'yolov5'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 166 (delta 53), reused 66 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (166/166), 986.98 KiB | 12.18 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/yolov5


In [ ]:
# ✅ STEP 4: Download & unzip datasets
!kaggle datasets download -d ankanghosh651/object-detection-wildlife-dataset-yolo-format
!unzip -q object-detection-wildlife-dataset-yolo-format.zip -d /content/wildlife

!kaggle datasets download -d trainingdatapro/miners-detection
!unzip -q miners-detection.zip -d /content/miners

Dataset URL: https://www.kaggle.com/datasets/ankanghosh651/object-detection-wildlife-dataset-yolo-format
License(s): MIT
 95% 426M/449M [00:01<00:00, 351MB/s]
100% 449M/449M [00:01<00:00, 423MB/s]
replace /content/wildlife/final_data/data_wl.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Dataset URL: https://www.kaggle.com/datasets/trainingdatapro/miners-detection
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
  0% 0.00/5.53M [00:00<?, ?B/s]
100% 5.53M/5.53M [00:00<00:00, 268MB/s]
replace /content/miners/annotations.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
# ✅ STEP 5: Combine datasets
import os, shutil
from glob import glob
import xml.etree.ElementTree as ET

!mkdir -p /content/combined_dataset/images/train
!mkdir -p /content/combined_dataset/labels/train

def copy_imgs(src, dst):
    for path in glob(f"{src}/*"):
        if path.endswith(('.jpg', '.png')):
            shutil.copy(path, dst)

def remap_labels(src, dst, offset=0):
    for label_path in glob(f"{src}/*.txt"):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls_id = int(parts[0]) + offset
                new_lines.append(f"{cls_id} {' '.join(parts[1:])}\n")
        with open(os.path.join(dst, os.path.basename(label_path)), 'a') as f:
            f.writelines(new_lines)

wildlife_imgs = glob("/content/wildlife/**/images", recursive=True)[0]
wildlife_lbls = glob("/content/wildlife/**/labels", recursive=True)[0]
copy_imgs(wildlife_imgs, "/content/combined_dataset/images/train")
remap_labels(wildlife_lbls, "/content/combined_dataset/labels/train")

miners_img_dir = "/content/miners/boxes"
miners_lbl_dir = "/content/miners/labels"
os.makedirs(miners_lbl_dir, exist_ok=True)

def convert_xml_to_yolo(xml_file, img_folder, label_output, class_id):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find("size")
    if size is None: return
    width = int(size.find("width").text)
    height = int(size.find("height").text)
    label_path = os.path.join(label_output, root.find("filename").text.rsplit(".", 1)[0] + ".txt")
    with open(label_path, "w") as f:
        for obj in root.findall("object"):
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            xmax = int(bbox.find("xmax").text)
            ymin = int(bbox.find("ymin").text)
            ymax = int(bbox.find("ymax").text)
            x_c = ((xmin + xmax) / 2) / width
            y_c = ((ymin + ymax) / 2) / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")

for ann in glob("/content/miners/*.xml"):
    convert_xml_to_yolo(ann, miners_img_dir, miners_lbl_dir, class_id=4)

copy_imgs(miners_img_dir, "/content/combined_dataset/images/train")
remap_labels(miners_lbl_dir, "/content/combined_dataset/labels/train", offset=0)


In [ ]:
# ✅ STEP 6: Create data.yaml
yaml_text = """
path: /content/combined_dataset
train: images/train
val: images/train

names:
  0: buffalo
  1: elephant
  2: rhino
  3: zebra
  4: miner
"""
with open("/content/yolov5/data/data.yaml", "w") as f:
    f.write(yaml_text)

In [ ]:
# ✅ Patch PyTorch 2.6+ to allow YOLOv5 model unpickling
from models.yolo import Model
import torch.serialization
torch.serialization.add_safe_globals({'Model': Model})


In [ ]:
!python train.py --img 640 --batch 16 --epochs 80 --data data/data.yaml --weights yolov5m.pt --name tuned_model --hyp data/hyps/hyp.scratch-low.yaml


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-03 18:12:13.498960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751566333.533922   20342 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751566333.549072   20342 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: tavi1117p (tavi1117p-netaji-subhas-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=, data=data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=16

In [ ]:
# STEP 8️⃣: Display results
import glob
from IPython.display import Image, display

output_dirs = sorted(glob.glob('/content/yolov5/runs/train/tuned_model*'), key=os.path.getmtime)
if output_dirs:
    last_run = output_dirs[-1]
    print(f"✅ Training logs in: {last_run}")
    result_imgs = glob.glob(f"{last_run}/*.jpg") + glob.glob(f"{last_run}/*.png")
    if result_imgs:
        display(Image(result_imgs[0]))
    else:
        print("⚠️ No result images found.")
else:
    print("❌ No output directory found.")


✅ Training logs in: /content/yolov5/runs/train/tuned_model2
⚠️ No result images found.


In [ ]:
# 🧠 Check where YOLO actually saved the model
import glob

model_paths = glob.glob('/content/yolov5/runs/train/*/weights/best.pt')
print("📦 Found model weights at:")
for path in model_paths:
    print(path)


📦 Found model weights at:


In [ ]:
# ✅ STEP 10: Download Trained Weights (auto-path)
from google.colab import files
import glob

model_paths = glob.glob('/content/yolov5/runs/train/*/weights/best.pt')
if model_paths:
    latest_model = sorted(model_paths, key=os.path.getmtime)[-1]
    print(f"📦 Found model weights at: {latest_model}")
    files.download(latest_model)
else:
    print("❌ No model weights found! Please ensure training completed without error.")



❌ No model weights found! Please ensure training completed without error.
